# Create silver table - quote

In [0]:
%run "../../setup/functions"

## Step 1 -Read Quote from Bronze using spark dataframe reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, BooleanType, DateType

In [0]:
df = spark.read.table("<your-catalog>.bronze.quote_1")

In [0]:
display(df.limit(100))

## Clean data

In [0]:
# Remove duplicate and nulls in the primary column: Id
semi_clean_df = remove_duplicates_and_nulls(df, "Id")

In [0]:
from pyspark.sql.functions import col

# Cast createdDate into DateType
clean_df = semi_clean_df.withColumn("CreatedDate", col("CreatedDate").cast(DateType()))

In [0]:
# Select and rename columns
quote_df = clean_df.select(
    col("Id").alias("id"),
    col("Name").alias("name"),
    col("CreatedDate").alias("created_date"),
    col("SBQQ__Account__c").alias("account_id"),
    col("SBQQ__ContractingMethod__c").alias("contracting_method"),
    col("SBQQ__CustomerDiscount__c").alias("customer_discount"),
    col("SBQQ__DaysQuoteOpen__c").alias("days_quote_open"),
    col("SBQQ__EndDate__c").alias("end_date"),
    col("SBQQ__ExpirationDate__c").alias("expiration_date"),
    col("SBQQ__Opportunity2__c").alias("opportunity_id"),
    col("SBQQ__PaymentTerms__c").alias("payment_terms"),
    col("SBQQ__ShippingCity__c").alias("shipping_city"),
    col("SBQQ__ShippingCountry__c").alias("shipping_country"),
    col("SBQQ__ShippingPostalCode__c").alias("shipping_postal_code"),
    col("SBQQ__ShippingState__c").alias("shipping_state"),
    col("SBQQ__StartDate__c").alias("start_date"),
    col("SBQQ__Status__c").alias("status"),
    col("SBQQ__SubscriptionTerm__c").alias("subscription_term"),
    col("SBQQ__CustomerAmount__c").alias("customer_amount"),
    col("SBQQ__LineItemCount__c").alias("line_item_count"),
    col("SBQQ__ListAmount__c").alias("list_amount"),
    col("SBQQ__NetAmount__c").alias("net_amount"),
    col("SBQQ__RegularAmount__c").alias("regular_amount"),
    col("ApprovalStatus__c").alias("approval_status"),
    col("CPQ_Additional_Discount__c").alias("additional_discount"),
    col("CPQ_Annual_Recurring_Revenue__c").alias("annual_recurring_revenue"),
    col("CPQ_Conga_Currency_Locale__c").alias("conga_currency_locale"),
    col("CPQ_Currency_Conversion_Rate__c").alias("currency_conversion_rate"),
    col("CPQ_First_Year_Value__c").alias("first_year_value"),
    col("CPQ_MOSAIQ_Package_Name_for_Output__c").alias("mosaiq_package_name_for_output"),
    col("SBQQ__Primary__c").alias("is_primary")
)

In [0]:
# Add ingestion column
final_df = add_ingestion_date(quote_df)

In [0]:
display(final_df.limit(100))

In [0]:
%sql
USE CATALOG <your-catalog>

In [0]:
final_df.write.mode("overwrite").saveAsTable("silver.quote")

In [0]:
dbutils.notebook.exit("Success")